import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=1882
cls_num=7
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=InlineSkate0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (100, 1883)
first_test_shape= (550, 1883)
classes_quantity= 7
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7}
Count_labels= [ 9 14 18 18 16 14 11]
max(train_feature_Altitude)= 7.0
min(train_feature_Altitude)= -2.2635
first_train_sample=
 [ 2.       -0.036166 -0.15164  ... -0.12786  -0.19202  -0.11038 ]
1 9	2 14	3 18	4 18	5 16	6 14	7 11	

In [3]:
np.shape(ecg)

(100, 1883)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 9	2 14	3 18	4 18	5 16	6 14	7 11	
 max = 18


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (9, 1883)
Up to class  2 train shape =  (23, 1883)
Up to class  3 train shape =  (41, 1883)
Up to class  4 train shape =  (59, 1883)
Up to class  5 train shape =  (75, 1883)
Up to class  6 train shape =  (89, 1883)
Up to class  7 train shape =  (100, 1883)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0  9 14 18 18 16 14 11]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  3.477399999999998
min magnitude class 1  =  -2.5181999999999993
after normalizing >>
max magnitude class 1  =  0.7389289392378988
min magnitude class 1  =  -1.0771490882655848

 cls 2  >> 
max magnitude class 2  =  3.407200000000001
min magnitude class 2  =  -2.3568000000000002
after normalizing >>
max magnitude class 2  =  0.7176652329314843
min magnitude class 2  =  -1.0282607378687836

 cls 3  >> 
max magnitude class 3  =  3.340299999999999
min magnitude class 3  =  -2.4181999999999992
after normalizing >>
max magnitude class 3  =  0.6974011025625495
min magnitude class 3  =  -1.0468589083419158

 cls 4  >> 
max magnitude class 4  =  3.4306
min magnitude class 4  =  -2.2704999999999997
after normalizing >>
max magnitude class 4  =  0.7247531350336227
min magnitude class 4  =  -1.0021203125946572

 cls 5  >> 
max magnitude class 5  =  3.6146999999999987
min magnitude class 5  =  -2.0925999999999987
after normalizing >>
max magnitude class 5  =  

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.07736111952505
max tst = 0.8448839886108925


In [14]:
len(wndws_test)

550

In [15]:
np.shape(wndws_test)

(550, 1883)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(9, 1883)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (100, 1882)
y_train =>  (100,)
X_test  =>  (550, 1882)
y_test  =>  (550,)
X_valid  =>  (100, 1882)
y_valid  =>  (100,)


y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [23]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
5/5 [==============================] - 3s 65ms/step - loss: 2.5014 - accuracy: 0.1400 - val_loss: 1.9880 - val_accuracy: 0.1800
Epoch 2/1000
5/5 [==============================] - 0s 25ms/step - loss: 1.9674 - accuracy: 0.1700 - val_loss: 1.8887 - val_accuracy: 0.2200
Epoch 3/1000
5/5 [==============================] - 0s 25ms/step - loss: 1.9108 - accuracy: 0.2400 - val_loss: 1.8498 - val_accuracy: 0.2600
Epoch 4/1000
5/5 [==============================] - 0s 26ms/step - loss: 1.7944 - accuracy: 0.2500 - val_loss: 1.7175 - val_accuracy: 0.2800
Epoch 5/1000
5/5 [==============================] - 0s 27ms/step - loss: 1.7807 - accuracy: 0.2600 - val_loss: 1.6333 - val_accuracy: 0.3200
Epoch 6/1000
5/5 [==============================] - 0s 26ms/step - loss: 1.6837 - accuracy: 0.2800 - val_loss: 1.5484 - val_accuracy: 0.3600
Epoch 7/1000
5/5 [==============================] - 0s 26ms/step - loss: 1.6163 - accuracy: 0.3500 - val_loss: 1.4451 - val_accuracy: 0.4000
Epoch 8/1000


Epoch 59/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 60/1000
5/5 [==============================] - 0s 20ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 61/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 62/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 63/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 64/1000
5/5 [==============================] - 0s 19ms/step - loss: 1.2411 - accuracy: 0.8400 - val_loss: 0.9391 - val_accuracy: 0.7500
Epoch 65/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.3102 - accuracy: 0.9200 - val_loss: 0.0139 - val_accuracy: 1.0000
Epoch 

In [24]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [25]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

18/18 [==============================] - 0s 5ms/step
array([[13, 10, 14,  6,  8,  2,  0],
       [ 1, 26, 25, 15, 10,  3,  6],
       [ 2, 11, 26, 13, 18,  4, 11],
       [ 0,  7,  9, 34, 21,  9, 10],
       [ 0,  3, 15, 26, 36, 12,  9],
       [ 0,  3,  6, 16, 20, 29, 10],
       [ 0,  0,  1, 11, 22,  1, 16]], dtype=int64)
              precision    recall  f1-score   support

           1       0.81      0.25      0.38        53
           2       0.43      0.30      0.36        86
           3       0.27      0.31      0.29        85
           4       0.28      0.38      0.32        90
           5       0.27      0.36      0.31       101
           6       0.48      0.35      0.40        84
           7       0.26      0.31      0.28        51

    accuracy                           0.33       550
   macro avg       0.40      0.32      0.33       550
weighted avg       0.38      0.33      0.33       550



In [26]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
5/5 [==============================] - 1s 82ms/step - loss: 3.2347 - accuracy: 0.1300 - val_loss: 2.0121 - val_accuracy: 0.2200
Epoch 2/1000
5/5 [==============================] - 0s 39ms/step - loss: 2.0018 - accuracy: 0.2100 - val_loss: 1.9050 - val_accuracy: 0.2800
Epoch 3/1000
5/5 [==============================] - 0s 29ms/step - loss: 1.9159 - accuracy: 0.2100 - val_loss: 1.7811 - val_accuracy: 0.2600
Epoch 4/1000
5/5 [==============================] - 0s 40ms/step - loss: 1.8231 - accuracy: 0.2000 - val_loss: 1.7588 - val_accuracy: 0.3300
Epoch 5/1000
5/5 [==============================] - 0s 41ms/step - loss: 1.7216 - accuracy: 0.2700 - val_loss: 1.5805 - val_accuracy: 0.3400
Epoch 6/1000
5/5 [==============================] - 0s 40ms/step - loss: 1.7008 - accuracy: 0.3000 - val_loss: 1.6037 - val_accuracy: 0.3600
Epoch 7/1000
5/5 [==============================] - 0s 38ms/step - loss: 1.6229 - accuracy: 0.3400 - val_loss: 1.4487 - val_accuracy: 0.3700
Epoch 8/1000


Epoch 59/1000
5/5 [==============================] - 0s 29ms/step - loss: 9.3579e-04 - accuracy: 1.0000 - val_loss: 6.3009e-04 - val_accuracy: 1.0000
Epoch 60/1000
5/5 [==============================] - 0s 28ms/step - loss: 6.4024e-04 - accuracy: 1.0000 - val_loss: 4.3103e-04 - val_accuracy: 1.0000
Epoch 61/1000
5/5 [==============================] - 0s 28ms/step - loss: 4.2018e-04 - accuracy: 1.0000 - val_loss: 3.1258e-04 - val_accuracy: 1.0000
Epoch 62/1000
5/5 [==============================] - 0s 29ms/step - loss: 3.2942e-04 - accuracy: 1.0000 - val_loss: 2.1376e-04 - val_accuracy: 1.0000
Epoch 63/1000
5/5 [==============================] - 0s 28ms/step - loss: 2.1837e-04 - accuracy: 1.0000 - val_loss: 1.4295e-04 - val_accuracy: 1.0000
Epoch 64/1000
5/5 [==============================] - 0s 28ms/step - loss: 1.4978e-04 - accuracy: 1.0000 - val_loss: 1.0560e-04 - val_accuracy: 1.0000
Epoch 65/1000
5/5 [==============================] - 0s 28ms/step - loss: 1.2253e-04 - accuracy: 1.0

In [27]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

18/18 [==============================] - 0s 8ms/step
array([[19,  9,  7,  3, 12,  3,  0],
       [ 7, 35, 23,  8,  6,  7,  0],
       [ 4, 12, 21,  6, 19, 12, 11],
       [ 0, 10, 14, 25, 22, 13,  6],
       [ 0,  3, 20, 21, 26, 22,  9],
       [ 0,  3,  3, 15, 12, 39, 12],
       [ 1,  2,  0,  5, 20, 10, 13]], dtype=int64)
              precision    recall  f1-score   support

           1       0.61      0.36      0.45        53
           2       0.47      0.41      0.44        86
           3       0.24      0.25      0.24        85
           4       0.30      0.28      0.29        90
           5       0.22      0.26      0.24       101
           6       0.37      0.46      0.41        84
           7       0.25      0.25      0.25        51

    accuracy                           0.32       550
   macro avg       0.35      0.32      0.33       550
weighted avg       0.34      0.32      0.33       550

